In [1]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import kaggle_xgb
import calc_ndcg
#from sklearn.grid_search import ParameterGrid
import pickle
import fetch_data
reload(fetch_data)

<module 'fetch_data' from 'fetch_data.pyc'>

Read in just the testing and training data.

In [2]:
xgbInput = fetch_data.clfInput()
xgbInput.sessions_ftrEng()
xgbInput.users_ftrEng()
xgbInput.one_hot()
#xgbInput.binarize_targets()
#xgbInput.split_data()

number of columns before one hot encoding 273
number of columns after one hot encoding 417


In [ ]:
param = {'num_class': 12, 'objective': 'multi:softprob', 'seed': 0}
param['eta'] = 0.20
param['max_depth'] = 6
param['subsample'] = .5
param['col_sample_bytree'] = .6
results = {}
cv_train = pd.read_pickle('cv_results/actions_e20/train_err.p')
cv_valid = pd.read_pickle('cv_results/actions_e20/validate_err.p')
nrounds = 40
for train_indx, valid_indx in cv_bydate(xgbInput):
    dtrain = xgb.DMatrix(xgbInput.train_X[train_indx], label = xgbInput.train_Y[train_indx],
                missing = -1)
    dvalid = xgb.DMatrix(xgbInput.train_X[valid_indx], label = xgbInput.train_Y[valid_indx],
                missing = -1)
    evallist = [(dtrain, 'train'), (dvalid, 'eval')]
    bst = xgb.train(param, dtrain, nrounds, evallist, feval = calc_ndcg.evalerror, evals_result = results)
    cv_train = pd.concat([cv_train, pd.Series(results['train']['error'])], axis = 1)
    cv_valid = pd.concat([cv_valid, pd.Series(results['eval']['error'])], axis = 1)
    pd.to_pickle(cv_train, 'cv_results/actions_e20/train_err.p')
    pd.to_pickle(cv_valid, 'cv_results/actions_e20/validate_err.p')

In [ ]:
def cv_sessions(xgbInput):
    """Cross-validation on a subset of training data which meets a condition"""
    
    condition = 'dac_year == 2014 & action_counts != -1'
    idx = np.nonzero(xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index))
    idx = idx[0]
    np.random.shuffle(idx[0])
    n = len(idx)
    
    for i in range(0, kfolds):
        idx[0]
        
def cv_bydate(xgbInput):
    """Select folds for cross validation as all cases that occurred in a given with month 
    in 2014, with sessions data.
    Only cases in 2014 have sessions data and the last test case is on June 30.
    """
    
    for i in range(4,7):
        condition = 'dac_year == 2014 & dac_month == @i & action_counts != -1'
        valid_mask = xgbInput.trainDf.index.isin(xgbInput.trainDf.query(condition).index)
        valid_indx = np.where(valid_mask)[0]
        train_indx = np.where(~valid_mask)[0]
        yield train_indx, valid_indx

In [20]:
pd.read_pickle('cv_results/sessions_e20_25n/validate_err.p')

,0,0,0,0,0,0,mean
0,0.14313,0.14935,0.14903,0.15058,0.15351,0.14546,0.148510
1,0.14245,0.14757,0.14722,0.15041,0.15296,0.14471,0.147553
2,0.14170,0.14719,0.14673,0.15047,0.15271,0.14427,0.147178
3,0.14116,0.14690,0.14640,0.15036,0.15228,0.14423,0.146888
4,0.14091,0.14657,0.14640,0.14991,0.15244,0.14457,0.146800
5,0.14127,0.14653,0.14624,0.14932,0.15245,0.14477,0.146763
6,0.14049,0.14589,0.14592,0.14909,0.15220,0.14497,0.146427
7,0.14028,0.14625,0.14579,0.14903,0.15172,0.14478,0.146308
8,0.14005,0.14609,0.14584,0.14867,0.15156,0.14471,0.146153
9,0.14063,0.14666,0.14540,0.14825,0.15164,0.14469,0.146212


In [18]:
a = pd.read_pickle('cv_results/actions_e20/validate_err.p')
a = a.astype(float)
a['mean'] = a.mean(axis = 1)

In [ ]:
null_tr_browsers = []
null_testbrowsers = []
for i in train_data.first_browser.unique():
    if i not in test_data.first_browser.unique():
        null_tr_browsers.append(i)
for i in test_data.first_browser.unique():
    if i not in train_data.first_browser.unique():
        null_testbrowsers.append(i)

In [ ]:
train_data[train_data.first_browser=='-unknown-'].first_browser.value_counts()

In [ ]:
train_data[train_data.affiliate_provider == "gsp"].country_destination.value_counts(normalize = True)

In [ ]:
print df.first_browser.value_counts()
print test_data.first_browser.value_counts()

In [ ]:
xgbInput.allDf.loc[xgbInput.allDf.query('1000 > age > 100').index, 'age'] = 105
#xgbInput.allDf.loc[self.allDf.query('age > 1000').index, 'age'] = 110

In [1]:
%run run_model.py

number of columns before one hot encoding 273
number of columns after one hot encoding 417


In [ ]:
np.where(av<14, -1)

In [ ]:
av = xgbInput.allDf.age.values

In [3]:
with open('../xgbmodels/actions_e20_26n.p') as f:
    bst = pickle.load(f)

In [4]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, xgbInput.allDf.columns[int(feat[1:])], score) for feat, score in features]

[('f0', 'age', 332),
 ('f257', 'dac_day', 186),
 ('f4', 'secs_elapsed', 185),
 ('f262', 'tfa_hour', 161),
 ('f3', 'action_counts', 129),
 ('f256', 'dac_month', 117),
 ('f37', 'ajax_refresh_subtotal', 105),
 ('f258', 'dac_dayofweek', 103),
 ('f1', 'booking_request', 98),
 ('f255', 'dac_year', 97),
 ('f227', 'this_hosting_reviews', 72),
 ('f267', 'signup_method_basic', 70),
 ('f155', 'personalize', 66),
 ('f2', 'message_post', 64),
 ('f201', 'search_results', 62),
 ('f274', 'signup_flow_3', 56),
 ('f212', 'similar_listings', 55),
 ('f263', 'gender_-unknown-', 49),
 ('f128', 'lookup', 48),
 ('f237', 'travel_plans_current', 46),
 ('f29', 'ajax_google_translate_reviews', 45),
 ('f356', 'first_device_type_Mac Desktop', 44),
 ('f348', 'first_affiliate_tracked_untracked', 44),
 ('f268', 'signup_method_facebook', 43),
 ('f260', 'tfa_month', 40),
 ('f179', 'qt2', 38),
 ('f78', 'dashboard', 38),
 ('f56', 'cancellation_policies', 35),
 ('f143', 'other_hosting_reviews_first', 35),
 ('f351', 'signup